In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from __future__ import division, print_function


In [2]:
df = pd.read_csv('../data/AB_NYC_2019/AB_NYC_2019.csv')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
# Target variable reviews_per_month reviews_per_month 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [4]:
# converting categorical features


def code_myohe(data, feature):
    for i in data[feature].unique():
        if type(i)==type(1.0):
            data[feature+"="+str(i)]=(data[feature]==i).astype("float64");
        else:
            data[feature+"="+i]=(data[feature]==i).astype("float64");
        
code_myohe(df, "neighbourhood_group");
df.drop(['neighbourhood_group'], axis=1, inplace=True) 
code_myohe(df, 'room_type');
df.drop(['room_type'], axis=1, inplace=True) 


#def code_myohe2(data, feature):   
#    j=0;
#    for i in data[feature].unique():
#        data[feature+"_num"]=j
#        j++;
#    categories_num = data[feature].unique().count
#    
#    for i in data[feature+"_num"].unique():
#        data[feature+"_e"+i]=j
#        j++;
        
lr = "last_review";
df[lr+"_weekday"] = pd.to_datetime(df[lr], errors='coerce').dt.strftime("%a");

In [5]:


code_myohe(df, lr+"_weekday");
df.drop([lr+"_weekday"], axis=1, inplace=True);
df[lr+'_month']= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%b");
code_myohe(df, lr+'_month');
df.drop([lr+'_month'], axis=1, inplace=True) ;
df[lr+'_day']= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%d");
code_myohe(df, lr+'_day');
df.drop([lr+'_day'], axis=1, inplace=True) ;
df[lr+"_year"]= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%Y");
code_myohe(df, lr+'_year');
df.drop([lr+'_year'], axis=1, inplace=True) ;
df.drop([lr], axis=1, inplace=True);
df.drop(["name"], axis=1, inplace=True);
df.drop(["host_name"], axis=1, inplace=True);
df.drop(["neighbourhood"], axis=1, inplace=True);

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 81 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 48895 non-null  int64  
 1   host_id                            48895 non-null  int64  
 2   latitude                           48895 non-null  float64
 3   longitude                          48895 non-null  float64
 4   price                              48895 non-null  int64  
 5   minimum_nights                     48895 non-null  int64  
 6   number_of_reviews                  48895 non-null  int64  
 7   reviews_per_month                  38843 non-null  float64
 8   calculated_host_listings_count     48895 non-null  int64  
 9   availability_365                   48895 non-null  int64  
 10  neighbourhood_group=Brooklyn       48895 non-null  float64
 11  neighbourhood_group=Manhattan      48895 non-null  flo

In [6]:
y = df['reviews_per_month']
y = y.fillna(-1)
y_classific = y>=(y.mean());
df.drop(['reviews_per_month'], axis=1, inplace=True)

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3, random_state=17)

y_regress_train, y_regress_holdout = y_train, y_holdout;
y_train, y_holdout = (y_train>=(y.mean())), (y_holdout>=(y.mean()));

tree = DecisionTreeClassifier(max_depth=6, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)
rndForest = RandomForestClassifier()
grdBost = GradientBoostingClassifier()

tree.fit(X_train, y_train)
knn.fit(X_train, y_train)
rndForest.fit(X_train, y_train)
grdBost.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [7]:
from sklearn.metrics import accuracy_score

zero_pred = np.zeros(shape=(y_holdout.shape));
print(accuracy_score(y_holdout, zero_pred)) # 0.636

tree_pred = tree.predict(X_holdout)
print(accuracy_score(y_holdout, tree_pred)) # 0.960

knn_pred = knn.predict(X_holdout)
print(accuracy_score(y_holdout, knn_pred)) # 0.664

rndForest_pred = rndForest.predict(X_holdout)
print(accuracy_score(y_holdout, rndForest_pred)) # 0.948

grdBost_pred = grdBost.predict(X_holdout)
print(accuracy_score(y_holdout, grdBost_pred)) # 0.966

0.6369213988683619
0.9601199809121276
0.6641216170154748
0.9511214125025564
0.9663917104097075


In [8]:

from sklearn.linear_model import LogisticRegression, LinearRegression

logRegress = LogisticRegression();

logRegress.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:


logRegress_pred = logRegress.predict(X_holdout)
print(accuracy_score(y_holdout, logRegress_pred)) # 0.652


0.652191696775513
